# Advanced Bayesian Inference with Case Studies
## Project: Salm: extra- Poisson variation in dose- response study

### 1. Project Topic: Bayesian Inference in Dose-Response for Mutagenicity Assay Data

#### Background:
Mutagenicity assays are used to evaluate the potential of chemical compounds to cause genetic mutations. In this project, we focus on the response of bacterial colonies to varying doses of a mutagenic agent, quinoline. The goal is to model the relationship between the dose of quinoline and the number of revertant colonies (mutant bacteria that have reverted to their original state).

#### Objective:
We aim to implement a Bayesian inference approach to model the Poisson variation in the dose-response data. By using Bayesian methods, we can estimate the parameters of the model and make probabilistic inferences about the effects of different doses on the mutagenicity assay results.

#### Model Assumptions:
1. $( y_{ij} \sim \text{Poisson}(\mu_{ij}) )$: The number of revertant colonies (observations) follows a Poisson distribution with mean $(\mu_{ij})$.
2. $( \log(\mu_{ij}) = \alpha + \beta \log(x_i + 10) + \gamma x_i + \lambda_{ij} )$: The log of the mean response is modeled as a linear combination of the log-transformed dose, the dose, and a random effect.
3. $( \lambda_{ij} \sim \text{Normal}(0, \tau) )$: The random effects follow a normal distribution with mean 0 and standard deviation $(\tau)$.

#### Parameters:
- $( \alpha )$, $( \beta )$, $( \gamma )$, and $( \tau )$ are given independent "noninformative" priors.

### 2. Plan
The project will be implemented in the following steps:
1. **Prepare the Data**:
   - In this step, we are going to create a dataset with doses and plate measurements.
   - We will organize the data into a suitable format for analysis.

2. **Define the Bayesian Model**:
   - In this step, we are going to use `pymc3` to implement the Bayesian model based on the given assumptions.
   - We will specify the priors for the parameters ($( \alpha )$, $( \beta )$, $( \gamma )$, and $( \tau )$).
   - We will define the likelihood function to model the observed data.

3. **Perform Bayesian Inference**:
   - In this step, we are going to use Markov Chain Monte Carlo (MCMC) methods to sample from the posterior distribution of the parameters.
   - We will run the sampling process and collect the results.

4. **Analyze the Results**:
   - In this step, we are going to visualize the parameter estimates using trace plots and summary statistics.
   - We will compare the Bayesian estimates with quasi-likelihood estimates reported in previous studies.

5. **Interpret the Findings**:
   - In this step, we are going to interpret the parameter estimates and their implications for the dose-response relationship.
   - We will discuss the advantages and limitations of the Bayesian approach in 



#### Step 1: Prepare the Data
##### Understand the Data Files

From the provided files:

- `salm.data.R` and `salm2.data.R`
  Define the observed data:
  - `y`: A 6 × 3 matrix of observed Poisson counts (responses).
  - `x`: A vector of 6 dose levels.
  - `Ndoses`: Number of dose levels (6).
  - `Nplates`: Number of plates per dose (3).

- `salm.init.R` and `salm2.init.R`
  Define initial values for MCMC:
  - `alpha`, `beta`, `gamma`, `tau`: Model parameters, initialized to 0 (except `tau = 0.1`).
  - `lambda`: A 6 × 3 matrix initialized to 0, representing the random effect.

- `post.R`
  - Reads posterior samples from `"samples.csv"`.
  - Uses `coda::summary(as.mcmc(post))` to analyze the MCMC results.
  - Checks for the `BUGSExamples` package and (if installed) attempts to run JAGS.


In [1]:
import numpy as np
import pandas as pd

# Data for doses and plates (as provided in salm.data.R)
doses = [0, 10, 33, 100, 333, 1000]
plates = [
    [15, 16, 16],
    [27, 33, 20],
    [21, 18, 26],
    [41, 38, 27],
    [29, 21, 33],
    [60, 41, 42]
]

df = pd.DataFrame(plates, columns=['Plate1', 'Plate2', 'Plate3'])
df['Dose'] = doses

doses = df['Dose'].values
plates = df[['Plate1', 'Plate2', 'Plate3']].values

In [2]:
init_values = {
    "alpha": 0.0,
    "beta": 0.0,
    "gamma": 0.0,
    "tau": 0.1,
    "lambda": np.zeros((6, 3))  # Random effects initialized to zero
}

print(init_values)

{'alpha': 0.0, 'beta': 0.0, 'gamma': 0.0, 'tau': 0.1, 'lambda': array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])}


In [5]:
df.to_csv("data\salm_data.csv", index=False)

In [6]:
init_df = pd.DataFrame(init_values["lambda"])
init_df.to_csv("data\salm_init.csv", index=False)

#### Step 3: Define the Bayesian Model